In [1]:
# 라이브러리 가져오기
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
# GPU 사용설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# parameter 설정
learning_rate = 0.001
num_epochs = 15
batch_size = 100

In [4]:
# 데이터셋 설정
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# dataloader 만들기
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# 학습모델 만들기
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(7*7*64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)

        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [7]:
# 모델 선언
model = CNN().to(device)

In [8]:
# Loss function & Optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Training
total_batch = len(data_loader)  # 전체 batch 사이즈 알 수 있음

for epoch in range(num_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    print('Epoch: {} Cost: {}'.format(epoch, avg_cost))
print("Finished")

Epoch: 0 Cost: 0.22343823313713074
Epoch: 1 Cost: 0.06068681553006172
Epoch: 2 Cost: 0.0449286550283432
Epoch: 3 Cost: 0.036711957305669785
Epoch: 4 Cost: 0.030538614839315414
Epoch: 5 Cost: 0.02590113878250122
Epoch: 6 Cost: 0.02207002602517605
Epoch: 7 Cost: 0.017783157527446747
Epoch: 8 Cost: 0.015273386612534523
Epoch: 9 Cost: 0.012349516153335571
Epoch: 10 Cost: 0.011067546904087067
Epoch: 11 Cost: 0.009607967920601368
Epoch: 12 Cost: 0.008677253499627113
Epoch: 13 Cost: 0.007080189883708954
Epoch: 14 Cost: 0.005936538800597191
Finished


In [10]:
# Test
with torch.no_grad():
    X_test = mnist_test.test_data.view(
        len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accurarcy: {}'.format(accuracy.item()))

Accurarcy: 0.982699990272522


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
